In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import numpy as np
import pandas as pd
import torch

from src.RAG_Calculater import RAG, get_top_n_articles
from src.Massege_Factory import massage_factory, massege_for_gbt
from src.Prompt_Factory import prompt_factory
from src.Case_Builder import (device,
                              bert_version,
                              bert_model_name,
                              genai_version,
                              genai_model_name,
                              prompt_strategy_used,
                              dataset_name,
                              massage_strategy
                              )

In [2]:
data_train = pd.read_json(f'src/dataset/clean/{dataset_name}/{bert_version}_train.json')
data_val = pd.read_json(f'src/dataset/clean/{dataset_name}/{bert_version}_validation.json')
data_test = pd.read_json(f'src/dataset/clean/{dataset_name}/{bert_version}_test.json')

In [3]:
data_train = data_train.loc[:4].copy()
data_val = data_val.loc[:4].copy()
data_test = data_test.loc[:4].copy()

In [4]:
data_train['rag_sentences'] = data_train['sentences_similarity'].apply(RAG)
data_val['rag_sentences'] = data_val['sentences_similarity'].apply(RAG)
data_test['rag_sentences'] = data_test['sentences_similarity'].apply(RAG)

In [ ]:
chatbot = pipeline("text-generation", model=genai_model_name, min_new_tokens=256, max_new_tokens=512, repetition_penalty=1.2, no_repeat_ngram_size=3)

results = []
summaries = []

In [ ]:
for idx in range(len(data_test)):
    print(f"\n {idx+1} / {len(data_test)}", end="")
    
    target_row = data_test.loc[idx]
    
    if massage_strategy == "few_shot": 
        ref_rows_indexes = get_top_n_articles(data_train['title_embedding'], target_row['title_embedding'], n=3)
        ref_rows = data_train.loc[ref_rows_indexes].reset_index(drop=True)
        
    else: 
        ref_rows = None
    
    prompt = prompt_factory(prompt_strategy_used, target_row, ref_rows)
    massage = [{"role": "user", "content": prompt}]
    summary = " ".join(data_test.loc[idx, 'summary'])
    # massage = massage_factory(massage_strategy, target_row, ref_rows)
    # if genai_version == "BioGBT":
    #     massage = massege_for_gbt(massage)
    # 
    # 
    # massage = [
    #     {"role": "user", "content": "You are a math expert. x=2, y=3."},
    #     {"role": "assistant", "content": "Great!"},
    #     {"role": "user", "content": "what is x+y?"}
    # ]
    answer = chatbot(massage)
    print(answer)
    
    if genai_version == "BioGBT":
        answer = answer[0]['generated_text'].split("## Answer:\n")[-1]
    else:
        answer = answer[0]['generated_text'][-1]['content']
        
    results.append(answer)
    summaries.append(summary)
    break
    

In [ ]:
results

In [ ]:
model_results = pd.DataFrame({
    'reference': summaries,
    'prediction': results
})

In [ ]:
#model_results.to_csv(f'results/{genai_version}_{massage_strategy}_summaries_{bert_version}_{dataset_name}_{prompt_strategy_used}_test.csv', index=False)

In [ ]:
model_results

In [ ]:
print(model_results.loc[0, 'prediction'])

In [ ]:
results[0]